# OOP (2024 Fall) HW1: 불변 순서쌍을 활용해 가변 순서쌍 만들기 
- 이름: 홍지민  
- 학번: 20210497

여기 홍길동, 99999999 대신 본인의 이름, 학번 작성

-----
## Part A: 불변 순서쌍 Pair
1. 클래스(또는 정적) 팩토리 메소드 `of`를 를 수정하여 완성하라.
    - https://velog.io/@cjh8746/%EC%A0%95%EC%A0%81-%ED%8C%A9%ED%86%A0%EB%A6%AC-%EB%A9%94%EC%84%9C%EB%93%9CStatic-Factory-Method
1. 주석 부분을 풀어 제너릭 클래스인 `Pair`가 제너릭 인터페이스 `Comparable`를 구현하도록 완성하라.
    - https://docs.oracle.com/en/java/javase/22/docs/api/java.base/java/lang/Comparable.html

In [32]:
record Pair< F extends Comparable<F>, S extends Comparable<S> >(F first, S second) implements Comparable<Pair<F, S>> {
    
    // 아래 팩토리 메소드 of가 컴파일되지 않고 있으니 컴파일되도록 수정하라 (힌트: 제너릭 메소드)
    static <F extends Comparable<F>, S extends Comparable<S>> Pair<F, S> of(F first, S second) {
        return new Pair<>(first, second);
    }
    
    
    // 사전순이 되도록 정의하기. 즉 first가 더 큰 쪽이 더 크고, first가 같으면 second를 비교
    @Override
    public int compareTo(Pair<F, S> pair) {
        int firstComparison = this.first().compareTo(pair.first());
        
        // first가 같을 경우 second를 비교
        if (firstComparison == 0) {
            return this.second().compareTo(pair.second());
        }
        return firstComparison;
    }
    
}

In [33]:
/* compareTo 가 잘 동작하는지
   - 왼쪽이 더 큰 경우
   - 왼쪽이 더 작은 경우
   - 같은 경우
   이런 경우를 각각 포함하도록 이렇게 최소 3개 이상의 테스트를 실행해 보라
*/
// String으로 비교
Pair<String, String> pair1 = Pair.of("test", "oop");
Pair<String, String> pair2 = Pair.of("modifiable", "pair");
Pair<String, String> pair3 = Pair.of("compair", "partb");

// 테스트 실행
System.out.println(pair1.compareTo(pair2));  // 결과: 1 ("test" > "modifiable")
System.out.println(pair3.compareTo(pair3));  // 결과: 0 (pair3과 pair3은 동일)
System.out.println(pair3.compareTo(pair2));  // 결과: -1 ("compair" < "modifiable")

// Integer로 비교
Pair<Integer, Integer> intPair1 = Pair.of(5, 1);
Pair<Integer, Integer> intPair2 = Pair.of(3 , 10);
Pair<Integer, Integer> intPair3 = Pair.of(3, 20);

// 테스트 실행
System.out.println(intPair1.compareTo(intPair2));  // 결과: 1 (5 > 3)
System.out.println(intPair3.compareTo(intPair3));  // 결과: 0 (intPair3과 intPair3은 동일)
System.out.println(intPair2.compareTo(intPair3));  // 결과: 0 (3 == 3, 10 < 20)

// 자료형이 다를 때 비교 (Integer, String)
Pair<Integer, String> mixedPair1 = Pair.of(5, "integer");
Pair<Integer, String> mixedPair2 = Pair.of(3 , "string");
Pair<Integer, String> mixedPair3 = Pair.of(3, "apple");

// 테스트 실행
System.out.println(mixedPair1.compareTo(mixedPair2)); // 결과: 1 (5 > 3)
System.out.println(mixedPair1.compareTo(mixedPair1)); // 결과: 0 (mixedPair1과 mixedPair1은 동일)
System.out.println(mixedPair3.compareTo(mixedPair2)); // 결과: -1 (3 == 3, 하지만 "apple" < "string")

7
0
-10
1
0
-1
1
0
-18


------
## Part B: 가변 순서쌍 ModifiablePair

불변 순서쌍만을 priviate 필드로 갖고 있는 가변 순서쌍 제너릭 클래스를 작성하라

가변 순서쌍은 불변 순서쌍과 마찬가지 이름의 getter 메소드가 있고 추가로 setter 메소드도 있다.

또, 가변 순서쌍도 불변 순서쌍과 마찬가지로 Comparable 제너릭 인터페이스를 구현하라.


In [42]:
class ModifiablePair<F extends Comparable<F>, S extends Comparable<S>> implements Comparable<ModifiablePair<F, S>> {
    private Pair<F,S> pair; // 이 field 말고 다른 field는 추가로 정의하지 말 것!

    ModifiablePair(F first, S second) {
        this.pair = Pair.of(first, second);
    }
    
    // getter들은 pair에게 위임(delegate)하는 방식으로 작성하라
    public F first() {
        return pair.first();
    }
    public S second() {
        return pair.second();
    }
    
    // setter들은 적절히 정의하라
    public void setFirst(F first) {
        this.pair = Pair.of(first, this.pair.second());
    }
    public void setSecond(S second) {
        this.pair = Pair.of(this.pair.first(), second);
    }

    
    // compareTo는 pair에게 위임(delegate)하는 방식으로 작성하라
    @Override
    public int compareTo(ModifiablePair<F, S> other) {
        return this.pair.compareTo(other.pair);
    }
    

    @Override
    public String toString() { 
        return pair.toString(); // toString은 그냥 pair에게 위임
    }
}

In [43]:
// ModifiablePair<F,S>의 getter와 setter들이 잘 동작하는지 확인할 수 있는 적절한 테스트 코드 작성하여 실행하라.
// 그러니까 `first()`, `second()`, `setFrist(F)`, `setSecond(S)` 메소드를 모두 활용하는 예시 코드를 작성해 실행하라는 말이다.

ModifiablePair pair = new ModifiablePair<String, Integer>("modifiablePair", 20);
//first(), second()메서드 사용
System.out.println("first: " + pair.first());
System.out.println("second: " + pair.second());

//setter메서드 사용
pair.setFirst("compair");
pair.setSecond(30);
System.out.println("first: " + pair.first());
System.out.println("second: " + pair.second());


first: modifiablePair
second: 20
first: compair
second: 30


In [45]:
/* 가변 순써상에 대해서도
  compareTo 가 잘 동작하는지
   - 왼쪽이 더 큰 경우
   - 왼쪽이 더 작은 경우
   - 같은 경우
   이런 경우를 각각 포함하도록 이렇게 최소 3개 이상의 테스트를 실행해 보라
*/

// String으로 비교
ModifiablePair<String, String> pair1 = new ModifiablePair<>("test", "oop");
ModifiablePair<String, String> pair2 = new ModifiablePair<>("modifiable", "pair");
ModifiablePair<String, String> pair3 = new ModifiablePair<>("compair", "partb");

// 테스트 실행
System.out.println(pair1.compareTo(pair2));  // 결과: 1 ("test" > "modifiable")
System.out.println(pair3.compareTo(pair3));  // 결과: 0 (pair3과 pair3은 동일)
System.out.println(pair3.compareTo(pair2));  // 결과: -1 ("compair" < "modifiable")

// Integer로 비교
ModifiablePair<Integer, Integer> intPair1 = new ModifiablePair<>(5, 1);
ModifiablePair<Integer, Integer> intPair2 = new ModifiablePair<>(3 , 10);
ModifiablePair<Integer, Integer> intPair3 = new ModifiablePair<>(3, 20);

// 테스트 실행
System.out.println(intPair1.compareTo(intPair2));  // 결과: 1 (5 > 3)
System.out.println(intPair3.compareTo(intPair3));  // 결과: 0 (intPair3과 intPair3은 동일)
System.out.println(intPair2.compareTo(intPair3));  // 결과: 0 (3 == 3, 10 < 20)

// 자료형이 다를 때 비교 (Integer, String)
ModifiablePair<Integer, String> mixedPair1 = new ModifiablePair<>(5, "integer");
ModifiablePair<Integer, String> mixedPair2 = new ModifiablePair<>(3 , "string");
ModifiablePair<Integer, String> mixedPair3 = new ModifiablePair<>(3, "apple");

// 테스트 실행
System.out.println(mixedPair1.compareTo(mixedPair2)); // 결과: 1 (5 > 3)
System.out.println(mixedPair1.compareTo(mixedPair1)); // 결과: 0 (mixedPair1과 mixedPair1은 동일)
System.out.println(mixedPair3.compareTo(mixedPair2)); // 결과: -1 (3 == 3, 하지만 "apple" < "string")

7
0
-10
1
0
-1
1
0
-18
